# Interactive seq2seq
Applying our trained encoder-decoder model so that it can produce and respond to dialogue in context. We will need to load the model, seq2seq_baseline.ckpt, which was trained on the NLP cluster with 2500 iterations and batch size of 1028.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
import random
import json
from tensorflow.python.layers.core import Dense

In [2]:
import sys
sys.path.append("../src/models/")
sys.path.append('../src/models/agents/')
sys.path.append('../src/data/')
from agent import Agent
from parse import FBParser

### Set global variables


In [3]:
train_iterations = 3
learning_rate = 0.1
max_input_length = 6 # length of goals list
max_output_length = 20
unk_threshold = 20

### Initialize parser and agent
Using the Agent() class which subclasses the basic TfEncoderDecoder for dialogue.

In [4]:
parser = FBParser(unk_threshold=unk_threshold,
                  input_directory="../data/raw/",
                  output_directory="../data/processed/")
print("Vocab size: {}".format(parser.vocab_size))

Vocab size: 502


In [12]:
agent = Agent(vocab=parser.vocab,
              max_iter=train_iterations,
              eta=learning_rate,
              max_input_length=max_input_length,
              max_output_length=max_output_length,
              hidden_dim=64)

### Model training

In [13]:
train_data = []
with open("../data/processed/train.txt", "r") as train_file:
    for line in train_file:
        train_example = json.loads(line)
        train_data.append((
            train_example["input"],
            train_example["output"][0].split()))

In [14]:
X, y = zip(*train_data)

In [15]:
agent.fit(X, y, save_path="../models/seq2seq")

Iteration 3: loss: 45.921010017395025

Model saved in path: ../models/seq2seq


Load the saved model:

In [28]:
tf.reset_default_graph()
a.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))

NameError: name 'self' is not defined

In [27]:
saver = tf.train.import_meta_graph('../models/seq2seq.meta')

In [23]:
saver.restore(agent.sess, '../models/seq2seq')

INFO:tensorflow:Restoring parameters from ../models/seq2seq


### Test data prediction
Make sure the saved model works correctly:

In [24]:
test_data = []
with open("../data/processed/test.txt", "r") as test_file:
    for line in test_file:
        test_example = json.loads(line)
        test_data.append((
            test_example["input"],
            test_example["output"][0].split()))

Get first twenty examples:

In [25]:
X_test, y_test = zip(*test_data[:20])

In [26]:
logits = agent.predict(X_test)
f.write('\nPredictions:\n')
for string in agent.output(logits, padding=" "):
    f.write(string)
    f.write("\n")

FailedPreconditionError: Attempting to use uninitialized value Variable_1
	 [[Node: Variable_1/read = Identity[T=DT_FLOAT, _class=["loc:@decoder_1/while/BasicDecoderStep/cond/embedding_lookup_1"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable_1)]]

Caused by op 'Variable_1/read', defined at:
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
 